In [1]:
import os
import glob
import shutil
from pathlib import Path

BPM-Detector Function:

In [2]:
import scipy
import numpy as np
import statistics
from scipy.signal import find_peaks
def find_relative_peaks(autocorr,sr):
    # Find peaks in the autocorrelation function
    peaks, _ = find_peaks(autocorr, height=.6,distance=int(sr*60/240),width=[0,12])#!?! may I need to raise or reduce? Do I need another option, like distance=217
    # Plot the autocorrelation function with detected peaks
    #plt.plot(autocorr)
    #plt.plot(peaks, autocorr[peaks], 'x', color='red')
    #plt.title('Autocorrelation Function with Peaks')
    #plt.xlabel('Lag')
    #plt.ylabel('Autocorrelation')
    #plt.show()
    return peaks

def estimate_bpm(leftds,sr):
    histo=[]
    n=18# number of tested frames
    #plt.plot(leftds)# plot whole song time series
    for est in range(n-1):#start a loop to extract bpm from n time frames
        core = leftds[int(len(leftds)*est/n):int(len(leftds)*est/n+sr/2)]
        # plt.plot(core)# plot core time series
        signal = leftds[int(len(leftds)*est/n):int(len(leftds)*est/n)+2*sr]
        #plt.plot(signal)# plot comparison signal time series
        autocorr = scipy.signal.correlate(core, signal, mode='valid')
        autocorr /= np.max(np.abs(autocorr))
        #print(len(autocorr))
        #lags = np.arange(len(autocorr))
        #plt.plot(lags, autocorr, label='Autocorrelation', color='green')
        relative_peaks = find_relative_peaks(autocorr,sr)
        #print(relative_peaks)
        bpm_hist=[]
        #lags_in_seconds = relative_peaks / 1378.13
        #print(lags_in_seconds)
        if len(relative_peaks)>1:
            for sub in range(len(relative_peaks)-1):
                beats=int(60*sr/(relative_peaks[sub+1]-relative_peaks[sub]))
                if beats < 90:
                    beats=beats*2
                if beats > 200:
                    beats=int(beats/2)
                #print(beats)
                histo.append(beats)
    mean_value = sum(histo) / len(histo)
# Define 60% and 140% of the mean
    low = 0.6 * mean_value
    up = 1.4 * mean_value
    modified_values = []
    for value in histo:
        if value < low:
            modified_values.append(value * 2)  # Double the value
        elif value < up:
            modified_values.append(value)  # Keep the value unchanged   
    #print(modified_values)
    return modified_values

Find CSV files to import, modify and export, and audio files to analyze:

In [3]:
#csvlist = []
csvpath = r'C:\Users\t_ziemer\Desktop\techno-csv\germany'#file path that contains all csv files
#for file in files:
#    csvlist.append(os.path.join(csvroot,file))

In [4]:
import librosa
import warnings
from matplotlib import pyplot as plt

mp3list = []
mp3names= []
targetpath = r'C:\Users\t_ziemer\Desktop\techno-test\germanycsv'
path = r'C:\Users\t_ziemer\Desktop\techno-test\Germany'#file path that contains all mp3 files to be analyzed
for root, dirs, files in os.walk(path):#loop to list all file paths
    for file in files:
        mp3list.append(os.path.join(root,file))
        mp3names.append(file)

check if you find the csv of the respective audio:

In [10]:
import pandas as pd
import csv
# Suppress DeprecationWarning
warnings.simplefilter("ignore", category=DeprecationWarning)
for n in range(len(mp3list)):#len(mp3list)
    while True:
        try:
            array, sr=librosa.load(mp3list[n],sr=500,mono=True)
            #print(f'round {n}')
            csvfile = pd.read_csv(os.path.join(csvpath,mp3names[n]+'.csv'),index_col=0)
            #print(list(csvfile['PhaseSpace']))
            #normalization:
            if np.max(array)>np.min(array)*-1:
                array = array/np.max(array)
            else:
                array = array/np.min(array)*-1
            #print('normalized')
            #print(sr)
            cut = array[(sr*4):len(array)-(sr*4)]# we crop the first and last 4 seconds
            #print('cut')
            estimated_bpm = estimate_bpm(cut,sr)
            #print('estimated')
            tempo = []
            tempo.append(statistics.mode(estimated_bpm))
            #print(tempo)
            if len(tempo) > 1:
                tempo = statistics.median(tempo)
            #print(tempo[0])
            beats=[]
            line=list(csvfile.index)
            #print(len(line))
            for i in range(len(line)):
                beats.append(int(tempo[0]))#bpm,median
            #print('lines added')
            print(tempo[0])
            #features=[np.transpose(line), np.transpose(bc), np.transpose(corr), np.transpose(peak), np.transpose(rms), np.transpose(crest), np.transpose(bclow), np.transpose(corrlow), np.transpose(peaklow), np.transpose(rmslow), np.transpose(crestlow), np.transpose(bcmid), np.transpose(corrmid), np.transpose(peakmid), np.transpose(rmsmid), np.transpose(crestmid), np.transpose(bchigh), np.transpose(corrhigh), np.transpose(peakhigh), np.transpose(rmshigh), np.transpose(cresthigh),np.transpose(beats)] 
            #features = [list(csvfile.index), list(csvfile['PhaseSpace']), list(csvfile['ChannelCorrelation']), list(csvfile['PeakMeter']), list(csvfile['RMS']), list(csvfile['CrestFactor']), list(csvfile['PhaseSpaceLow']), list(csvfile['ChannelCorrelationLow']), list(csvfile['PeakMeterLow']), list(csvfile['RMSLow']), list(csvfile['CrestFactorLow']), list(csvfile['PhaseSpaceMid']), list(csvfile['ChannelCorrelationMid']), list(csvfile['PeakMeterMid']), list(csvfile['RMSMid']), list(csvfile['CrestFactorMid']), list(csvfile['PhaseSpaceHigh']), list(csvfile['ChannelCorrelationHigh']), list(csvfile['PeakMeterHigh']), list(csvfile['RMSHigh']), list(csvfile['CrestFactorHigh']), np.transpose(beats)]
            #features = [list(csvfile.index), list(csvfile['PhaseSpace']), list(csvfile['ChannelCorrelation']), list(csvfile['PeakMeter']), list(csvfile['RMS']), list(csvfile['CrestFactor']), list(csvfile['PhaseSpaceLow']), list(csvfile['ChannelCorrelationLow']), list(csvfile['PeakMeterLow']), list(csvfile['RMSLow']), list(csvfile['CrestFactorLow']), list(csvfile['PhaseSpaceMid']), list(csvfile['ChannelCorrelationMid']), list(csvfile['PeakMeterMid']), list(csvfile['RMSMid']), list(csvfile['CrestFactorMid']), list(csvfile['PhaseSpaceHigh']), list(csvfile['ChannelCorrelationHigh']), list(csvfile['PeakMeterHigh']), list(csvfile['RMSHigh']), list(csvfile['CrestFactorHigh']), np.transpose(beats)]
            features=np.array([list(csvfile.index), list(csvfile['PhaseSpace']), list(csvfile['ChannelCorrelation']), list(csvfile['PeakMeter']), list(csvfile['RMS']), list(csvfile['CrestFactor']), list(csvfile['PhaseSpaceLow']), list(csvfile['ChannelCorrelationLow']), list(csvfile['PeakMeterLow']), list(csvfile['RMSLow']), list(csvfile['CrestFactorLow']), list(csvfile['PhaseSpaceMid']), list(csvfile['ChannelCorrelationMid']), list(csvfile['PeakMeterMid']), list(csvfile['RMSMid']), list(csvfile['CrestFactorMid']), list(csvfile['PhaseSpaceHigh']), list(csvfile['ChannelCorrelationHigh']), list(csvfile['PeakMeterHigh']), list(csvfile['RMSHigh']), list(csvfile['CrestFactorHigh']), beats])
            #print('asasas1')
            Details = ["", 'PhaseSpace', 'ChannelCorrelation','PeakMeter','RMS', 'CrestFactor', 'PhaseSpaceLow', 'ChannelCorrelationLow', 'PeakMeterLow', 'RMSLow', 'CrestFactorLow', 'PhaseSpaceMid', 'ChannelCorrelationMid', 'PeakMeterMid','RMSMid','CrestFactorMid', 'PhaseSpaceHigh', 'ChannelCorrelationHigh', 'PeakMeterHigh','RMSHigh','CrestFactorHigh','bpm']
            #Details = ["", 'PhaseSpace','bpm']
            print(os.path.join(targetpath,mp3names[n]+'.csv'))
            with open(os.path.join(targetpath,mp3names[n]+'.csv'), 'w', newline='') as f:
                #print('cool')
                write = csv.writer(f) 
                write.writerow(Details)
                print('cool')
                write.writerows(np.transpose(features)) 
            break      
        except:
            print('shit!')
            break

C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


128
C:\Users\t_ziemer\Desktop\techno-test\germanycsv\1000 Micro Dats - The Centre (Tuned Mix) (1993).m4a.csv
cool


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


138
C:\Users\t_ziemer\Desktop\techno-test\germanycsv\2Hard Out On Hi - One Good Nite On Hi (1988).m4a.csv
cool


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


...

200
C:\Users\t_ziemer\Desktop\techno-test\germanycsv\X-Cite - It Takes You (Renegade Mix) (1994).m4a.csv
cool


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


shit!
126
C:\Users\t_ziemer\Desktop\techno-test\germanycsv\XME - Automatic Lover (1993).m4a.csv
cool


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


109
C:\Users\t_ziemer\Desktop\techno-test\germanycsv\Ziggy P. - Work My Body Over (Part I) (1991).m4a.csv
cool


In [ ]:
# Example usage:
file_path = r'C:\Users\t_ziemer\Desktop\techno-test\germanytest\E-Noize - Descraciado (1993).m4a'
array, sr=librosa.load(file_path,sr=500,mono=True)
cut = array[(sr*4):len(array)-(sr*4)]
estimated_bpm = estimate_bpm(cut,sr)

median=np.median(estimated_bpm)
mode=statistics.mode(estimated_bpm)

if estimated_bpm is not None:
    print(f'Estimated BPM: {median} and {mode}')
else:
    print('BPM estimation failed.')